In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
# os.environ['XLA_FLAGS'] ='--xla_gpu_deterministic_ops=true'


os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".95"

import torch
torch.multiprocessing.set_start_method('spawn')

In [3]:
# Add parent folder to path (to run this file from subdirectories)
(parent_folder_path, current_dir) = os.path.split(os.path.abspath(''))
sys.path.append(parent_folder_path)

# add git submodule to path to allow imports to work
submodule_name = 'AlphaTrade'
sys.path.append(os.path.join(parent_folder_path, submodule_name))

print(sys.path)

from gymnax_exchange.jaxob.jorderbook import OrderBook
import gymnax_exchange.jaxob.JaxOrderBookArrays as job

['/data1/sascha/LOBS5/lob', '/data1/sascha/AlphaTrade', '/home/sascha/miniconda3/envs/alphatrade_new/lib/python310.zip', '/home/sascha/miniconda3/envs/alphatrade_new/lib/python3.10', '/home/sascha/miniconda3/envs/alphatrade_new/lib/python3.10/lib-dynload', '', '/home/sascha/miniconda3/envs/alphatrade_new/lib/python3.10/site-packages', '/data1/sascha/LOBS5', '/data1/sascha/LOBS5/AlphaTrade']


In [4]:
# from argparse import Namespace
from glob import glob
import numpy as onp
import pandas as pd
# from functools import partial
# from typing import Union, Optional
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
# from line_profiler import LineProfiler

import jax
import jax.numpy as jnp
from jax.nn import one_hot
# from jax import random
# from jax.scipy.linalg import block_diag
# from flax import jax_utils
# from flax.training import checkpoints
# import orbax

#from lob.lob_seq_model import BatchLobPredModel
# from lob.train_helpers import create_train_state, eval_step, prep_batch, cross_entropy_loss, compute_accuracy
from s5.ssm import *
# from s5.ssm_init import make_DPLR_HiPPO
# from s5.dataloading import make_data_loader
# from lob_seq_model import LobPredModel
from encoding import Vocab, Message_Tokenizer
# from lobster_dataloader import LOBSTER_Dataset, LOBSTER_Subset, LOBSTER_Sampler, LOBSTER

import preproc
# import inference
from lob import inference_no_errcorr as inference
import validation_helpers as valh
from lob.init_train import init_train_state, load_checkpoint, load_metadata, load_args_from_checkpoint
# import lob.encoding as encoding

# necessary for flax checkpoints to be loaded in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

from functools import partial



/tmp/ipykernel_772797/1832217467.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-08-05 10:40:46.262515: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 10:40:46.284185: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 10:40:46.290415: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable

# Forecasting

In [5]:
import lob.evaluation as eval
from preproc import transform_L2_state

In [6]:
n_gen_msgs = 1000  #500 # how many messages to generate into the future
n_messages_conditional = 500
n_eval_messages = 1000  # how many to load from dataset 
eval_seq_len = (n_eval_messages-1) * Message_Tokenizer.MSG_LEN
cond_seq_len = (n_messages_conditional) * Message_Tokenizer.MSG_LEN

data_levels = 10
# TODO: deprecated - remove from functions
sim_book_levels = 20 # 10  # order book simulator levels
sim_queue_len = 100  # per price in sim, how many orders in queue

n_vol_series = 500  # how many book volume series model uses as input

v = Vocab()
n_classes = len(v)
book_dim = 501 #b_enc.shape[1]
eval_book_seq_len = eval_seq_len


rng = jax.random.key(42)
rng, rng_ = jax.random.split(rng)
sample_top_n = 50
tick_size = 100

In [7]:
v.DECODING

{}

In [8]:
# entire test set after training data
stock='GOOG'


if stock == 'GOOG':
    data_dir = '/data1/sascha/data/GOOG2017to2019'
    ckpt_path='/data1/sascha/data/checkpoints/honest-oath-159_3kn3xbd5'
elif stock == 'INTC':
    raise NotImplementedError("Nothing trained for INTC yet")
elif stock == 'TSLA':
    raise Warning("Saved Model was trained on GOOGLE data. Generating for TSLA")
    data_dir = '/data1/sascha/data/lobster_proc'
    ckpt_path = '/data1/sascha/data/checkpoints/honest-oath-159_3kn3xbd5' # Dummy model trained on just 5 days... for debugging. 



In [9]:
args = load_metadata(ckpt_path)
ds = inference.get_dataset(data_dir, n_messages_conditional, n_eval_messages)
args.num_devices=1
args.bsz=1


new_train_state, model_cls = init_train_state(
    args,
    n_classes=n_classes,
    seq_len=eval_seq_len,
    book_dim=book_dim,
    book_seq_len=eval_book_seq_len,
)



ckpt = load_checkpoint(
    new_train_state,
    ckpt_path,
    train=False,
)
state = ckpt['model']


import chex
chex.clear_trace_counter()

Dummy input shapes (msg,book) ((Array(1, dtype=int32, weak_type=True), Array(21978, dtype=int32, weak_type=True)), 
 (Array(1, dtype=int32, weak_type=True), Array(21978, dtype=int32, weak_type=True), Array(501, dtype=int32, weak_type=True)))
(12012, 512)
configuring standard optimization setup
[*] Trainable Parameters: 25699641
(12012, 512)
(1, 12012, 512)


In [10]:
# jax.tree_util.tree_map(lambda x: x.shape,state)
ckpt = load_checkpoint(
    new_train_state,
    ckpt_path,
    train=False,
)
state = ckpt['model']
print(state.params['message_encoder']['encoder']['embedding'].shape)


(12012, 512)


In [11]:
len(ds.message_files)

743

In [12]:
import logging
# logging.basicConfig(filename='ar_debug.log', level=logging.DEBUG)
fhandler = logging.FileHandler(filename='generation_debug.log', mode='w')
logger = logging.getLogger()
if (logger.hasHandlers()):
    logger.handlers.clear()
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)
# logger.setLevel(logging.WARNING)

In [13]:
model = model_cls(training=False, step_rescale=1.0)


In [14]:
i = 200
l2_state_n = 20
# m_seq conditional (500) and eval messages (100) tokenised
# _ Target: scalar placeholder. 
# b_seq_pv raw form of book 4*nlvl+1
# msg_seq_raw msgs after preproc but before tokenising
# book_l2_init: n-1 book state... needs attention for unconditional case. 
m_seq, _, b_seq_pv, msg_seq_raw, book_l2_init = ds[int(i)]


#  b_seq : transformed into 501 size array. 
b_seq = jnp.array(transform_L2_state(b_seq_pv, n_vol_series, 100))
b_seq=jax.device_put(b_seq,device=jax.devices()[0])
m_seq_raw_inp = msg_seq_raw[: n_messages_conditional]

# with jax.disable_jit():

# initialise simulator
sim_init, sim_state_init = inference.get_sim(
    book_l2_init,  # book state before any messages
    m_seq_raw_inp, # messages to replay to init sim
)
# # book state after initialisation (replayed messages)
# l2_book_state_init = sim_init.get_L2_state(sim_state_init, l2_state_n)


m_seq=jnp.concatenate([jnp.array([v.START_TOK]),m_seq])

In [24]:
msg_seq_raw

inference.msg_to_lobster_format(msg_seq_raw).price.describe()

count    1.500000e+03
mean     7.880375e+06
std      2.200660e+04
min      7.840000e+06
25%      7.863800e+06
50%      7.864600e+06
75%      7.896000e+06
max      7.926200e+06
Name: price, dtype: float64

In [36]:
init_hidden=model.initialize_carry(args.bsz//args.num_devices,
                                        hidden_size=(args.ssm_size_base // pow(2,int(args.conj_sym))),
                                        n_message_layers=args.n_message_layers,
                                        n_book_pre_layers=args.n_book_pre_layers ,
                                        n_book_post_layers=args.n_book_post_layers,
                                        n_fused_layers=args.n_layers,)

In [81]:
msgs_decoded, l2_book_states, num_errors,msgs_tokens = inference.generate(
    sim_init,
    state,
    model,
    args.batchnorm,
    v.ENCODING,
    sample_top_n,
    tick_size,
    jax.device_put(jnp.array(m_seq[:11000+1])),#include start token
    jax.device_put(jnp.array(b_seq[:500])),
    1, #n_gen_msgs
    sim_state_init,
    rng,
    init_hidden,
    True,
    (0,0),
)

Best logits for index 0 after the mask: 
 [[1004 1006 1005 ...    2    1    0]]
Best logits for index 0 before the mask: 
 [[1004 1006 1005 ...  512  461  468]]
Mask for index 0: 
 [False False False ... False False False]
Best logits for index 1 after the mask: 
 [[12010 12011     3 ...     2     1     0]]
Best logits for index 1 before the mask: 
 [[12010 12011 12008 ...   278   603   285]]
Mask for index 1: 
 [False False False ... False  True  True]
Best logits for index 2 after the mask: 
 [[12009 12008 12011 ...     2     1     0]]
Best logits for index 2 before the mask: 
 [[12009 12008 12010 ...   859   285   603]]
Mask for index 2: 
 [False False False ...  True False False]
Best logits for index 3 after the mask: 
 [[11095 11093 11094 ...     2     1     0]]
Best logits for index 3 before the mask: 
 [[11095 11093 11094 ...   758    32    29]]
Mask for index 3: 
 [False False False ... False False False]
Best logits for index 4 after the mask: 
 [[1014 1032 1026 ...    2    1

In [82]:
inference.msg_to_lobster_format(msgs_decoded[:1])

,time,event_type,order_id,size,price,direction
0,25229.703371149,1,1,6,7900100,-1


In [83]:
msgs_decoded[:1]

Array([[        1,         1,         0,   7900100,       127,         6,
                0,   2204670,     25229, 703371149,         0,     -9999,
                0,         0]], dtype=int32)

In [103]:
msgs_tokens[:1]

Array([[ 1004, 12010, 12009, 11135,  1014,     4,     6,   208,   674,
           29,   233,   707,   375,   153, 12008, 11008,     2,     4,
            4,     4,     4,     4]], dtype=int32)

In [106]:
v.ENCODING['price']


from encoding import decode

print(decode(11008,*v.ENCODING['price']))
decode(0,*v.ENCODING['sign'])


0


Array(-10000, dtype=int32)

In [101]:
v.ENCODING['sign']

(Array([-10000, -20000,  -9999, -30000,     -1,      1], dtype=int32),
 Array([    0,     1,     2,     3, 12008, 12009], dtype=int32))

In [87]:
np.set_printoptions(edgeitems=5)
jnp.set_printoptions(edgeitems=5)

In [ ]:
11008, ..., 12003, 12004, 12005, 12006,12007

In [53]:
jax.device_put(jnp.array(m_seq[:11000+1]))#[:-1]
valh.get_field_from_idx(21)
valh.get_idx_from_field("time_ns")
valh.get_valid_toks_for_field(['time_ns'])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 jax.device_put(jnp.array(m_seq[:11000+1]))#[:-1]                                             │
│   2 valh.get_field_from_idx(21)                                                                  │
│   3 valh.get_idx_from_field("time_ns")                                                           │
│ ❱ 4 valh.get_valid_toks_for_field(['time_ns'])                                                   │
│   5                                                                                              │
│                                                                                                  │
│ /data1/sascha/LOBS5/lob/validation_helpers.py:114 in get_valid_toks_for_field                    │
│                                                                                                  │
│   111 def get_valid_toks_for_field(fields):                                                      │
│   112 │   """ Get the valid labels for given fields                                              │
│   113 │   """                                                                                    │
│ ❱ 114 │   return tuple(tuple(                                                                    │
│   115 │   │   v.DECODING[Message_Tokenizer.FIELD_ENC_TYPES[field]].keys())                       │
│   116 │   │     for field in fields)                                                             │
│   117                                                                                            │
│                                                                                                  │
│ /data1/sascha/LOBS5/lob/validation_helpers.py:115 in <genexpr>                                   │
│                                                                                                  │
│   112 │   """ Get the valid labels for given fields                                              │
│   113 │   """                                                                                    │
│   114 │   return tuple(tuple(                                                                    │
│ ❱ 115 │   │   v.DECODING[Message_Tokenizer.FIELD_ENC_TYPES[field]].keys())                       │
│   116 │   │     for field in fields)                                                             │
│   117                                                                                            │
│   118 def get_valid_toks_for_input(inp_maybe_batched):                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'time'

In [54]:
tuple(tuple(
        v.ENCODING[Message_Tokenizer.FIELD_ENC_TYPES[field]].keys())
          for field in ['time_s'])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 tuple(tuple(                                                                                 │
│   2 │   │   v.ENCODING[Message_Tokenizer.FIELD_ENC_TYPES[field]].keys())                         │
│   3 │   │     for field in ['time_s'])                                                           │
│   4                                                                                              │
│                                                                                                  │
│ in <genexpr>:2                                                                                   │
│                                                                                                  │
│   1 tuple(tuple(                                                                                 │
│ ❱ 2 │   │   v.ENCODING[Message_Tokenizer.FIELD_ENC_TYPES[field]].keys())                         │
│   3 │   │     for field in ['time_s'])                                                           │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'tuple' object has no attribute 'keys'

In [46]:
v.DECODING

{}

In [40]:
for field in ['time_s']:
    print(field)

time_s


In [23]:
inference.msg_to_lobster_format( msgs_decoded[0:10])


,time,event_type,order_id,size,price,direction
0,25229.703371149,1,1000,6,7900100,-1
1,25229.703405248,3,-1,6,7896000,-1
2,25229.704395401,1,998,18,7898500,-1
3,25229.704439751,3,-1,6,7898200,-1
4,25229.739425619,3,-1,6,7874600,1
5,25229.739440119,3,-1,60,7863200,1
6,25229.743440376,3,-1,6,7900100,-1
7,25229.745325586,1,993,6,7897600,-1
8,25229.745332256,1,992,24,7862700,1
9,25229.745348240,3,-1,6,7897700,-1


In [30]:
inference.book_to_lobster_format(l2_book_states[0:10,15:35])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,6,7926200,66,100,100,1999999900,100,-1,0,-1,0,-1,0,-1,0,-1,0,-1,0,-1
1,6,7926200,66,100,100,1999999900,100,-1,0,-1,0,-1,0,-1,0,-1,0,-1,0,-1
2,6,7916600,6,100,100,7926200,66,-1,0,1999999900,100,-1,0,-1,0,-1,0,-1,0,-1
3,6,7916600,6,100,100,7926200,66,-1,0,1999999900,100,-1,0,-1,0,-1,0,-1,0,-1
4,6,7916600,6,100,100,7926200,66,-1,0,1999999900,100,-1,0,-1,0,-1,0,-1,0,-1
5,6,7916600,6,100,100,7926200,66,-1,0,1999999900,100,-1,0,-1,0,-1,0,-1,0,-1
6,6,7916600,6,100,100,7926200,66,-1,0,1999999900,100,-1,0,-1,0,-1,0,-1,0,-1
7,6,7900100,6,100,100,7916600,6,-1,0,7926200,66,-1,0,1999999900,100,-1,0,-1,0,-1
8,24,7900100,6,7862400,6,7916600,6,100,100,7926200,66,-1,0,1999999900,100,-1,0,-1,0,-1
9,24,7900100,6,7862400,6,7916600,6,100,100,7926200,66,-1,0,1999999900,100,-1,0,-1,0,-1


In [20]:
n_samples = 160
batch_size = 16

# m_seq_gen, b_seq_gen, msgs_decoded, l2_book_states, num_errors = inference.sample_new(
# saves data to disk
inference.sample_new(
    n_samples,
    batch_size,
    ds,
    rng,
    cond_seq_len,
    n_eval_messages,
    n_gen_msgs,
    state,
    model,
    args.batchnorm,
    v.ENCODING,
    stock,
    save_folder='/data1/sascha/data/GOOG_eval',
    args=args,
    conditional= True,
    init_time = (jnp.array([0]),jnp.array([0])),
)

([(1, 1, 256), (1, 1, 256)], ([(1, 1, 256)], [(1, 1, 256)]), [(1, 1, 256), (1, 1, 256), (1, 1, 256), (1, 1, 256), (1, 1, 256), (1, 1, 256), (1, 1, 256), (1, 1, 256), (1, 1, 256), (1, 1, 256), (1, 1, 256), (1, 1, 256)])
([(16, 1, 1, 256), (16, 1, 1, 256)], ([(16, 1, 1, 256)], [(16, 1, 1, 256)]), [(16, 1, 1, 256), (16, 1, 1, 256), (16, 1, 1, 256), (16, 1, 1, 256), (16, 1, 1, 256), (16, 1, 1, 256), (16, 1, 1, 256), (16, 1, 1, 256), (16, 1, 1, 256), (16, 1, 1, 256), (16, 1, 1, 256), (16, 1, 1, 256)])
((1,), (1,))
((16, 1), (16, 1))


  0%|          | 0/10 [00:00<?, ?it/s]

BATCH [225109, 43405, 23400, 204158, 120045, 156096, 111123, 184358, 201905, 25776, 217364, 47568, 179108, 153087, 86083, 180859]
(16, 33001)
Shapes: (11000,) (11000, 501) (11000,) (11000,)
num_errors [488 530 642 533 392 516 434 581 606 497 596 533 607 641 537 461]


 10%|█         | 1/10 [01:13<11:05, 73.93s/it]

BATCH [30075, 71460, 235944, 227142, 76711, 173973, 144488, 148192, 197492, 163529, 164720, 105177, 74853, 3550, 163062, 212693]
(16, 33001)
num_errors [580 611 544 502 474 477 473 519 592 522 539 453 492 569 493 557]


...

 80%|████████  | 8/10 [07:05<01:41, 50.81s/it]

BATCH [25850, 23756, 75245, 234753, 145059, 16959, 52811, 213331, 159605, 63000, 159380, 67265, 226941, 20129, 64934, 50975]
(16, 33001)
num_errors [261 348 425 584 557 524 557 511 467 632 477 515 619 586 628 510]


 90%|█████████ | 9/10 [07:55<00:50, 50.44s/it]

BATCH [2942, 9521, 189484, 165953, 233652, 88681, 98372, 212118, 24167, 139837, 187348, 29207, 171535, 212651, 74332, 29392]
(16, 33001)
num_errors [465 363 528 481 460 586 519 512 574 446 427 547 616 488 498 500]


100%|██████████| 10/10 [08:46<00:00, 52.67s/it]
